In [ ]:
import flash
import mlflow
import torch
from flash.core.data.utils import download_data
from flash.text import TextClassificationData, TextClassifier

In [18]:
download_data("https://pl-flash-data.s3.amazonaws.com/imdb.zip", "./data/")
datamodule = TextClassificationData.from_csv(
    input_fields="review",
    target_fields="sentiment",
    train_file="data/imdb/train.csv",
    val_file="data/imdb/valid.csv",
    test_file="data/imdb/test.csv"
)

classifier_model = TextClassifier(backbone="prajjwal1/bert-tiny", num_classes=datamodule.num_classes)
trainer = flash.Trainer(max_epochs=3, gpus=torch.cuda.device_count())

Using custom data configuration default


 14%|█▎        | 3088/22500 [00:00<00:00, 30876.05ex/s]

Dataset csv downloaded and prepared to /Users/yongliu/.cache/huggingface/datasets/csv/default-383d5d0d91bc0c31/0.0.0/2960f95a26e85d40ca41a230ac88787f715ee3003edaacb8b1f0891e9f04dda2. Subsequent calls will reuse this data.


100%|██████████| 23/23 [00:05<00:00,  3.92ba/s]
Using custom data configuration default
0 tables [00:00, ? tables/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

Dataset csv downloaded and prepared to /Users/yongliu/.cache/huggingface/datasets/csv/default-239fb00a4c86d06b/0.0.0/2960f95a26e85d40ca41a230ac88787f715ee3003edaacb8b1f0891e9f04dda2. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00,  4.64ba/s]
Using custom data configuration default
0 tables [00:00, ? tables/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

Dataset csv downloaded and prepared to /Users/yongliu/.cache/huggingface/datasets/csv/default-1211bb345629d349/0.0.0/2960f95a26e85d40ca41a230ac88787f715ee3003edaacb8b1f0891e9f04dda2. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00,  4.57ba/s]
Using 'prajjwal1/bert-tiny' provided by Hugging Face/transformers (https://github.com/huggingface/transformers).
Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical

In [19]:

EXPERIMENT_NAME = "dl_model_chapter03"
mlflow.set_tracking_uri('http://localhost')
mlflow.set_experiment(EXPERIMENT_NAME)
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
print("experiment_id:", experiment.experiment_id)

experiment_id: 1


In [20]:
mlflow.pytorch.autolog()

with mlflow.start_run(experiment_id=experiment.experiment_id, run_name="chapter03") as dl_model_tracking_run:
    trainer.finetune(classifier_model, datamodule=datamodule, strategy="freeze")
    trainer.test()


  | Name          | Type                          | Params
----------------------------------------------------------------
0 | train_metrics | ModuleDict                    | 0     
1 | val_metrics   | ModuleDict                    | 0     
2 | model         | BertForSequenceClassification | 4.4 M 
----------------------------------------------------------------
258       Trainable params
4.4 M     Non-trainable params
4.4 M     Total params
17.545    Total estimated model params size (MB)


Testing: 100%|█████████▉| 623/625 [00:10<00:00, 59.74it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.6620000004768372, 'test_cross_entropy': 0.6180078387260437}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 625/625 [00:10<00:00, 57.96it/s]


In [21]:
run_id = dl_model_tracking_run.info.run_id
print("run_id: {}; lifecycle_stage: {}".format(run_id,
    mlflow.get_run(run_id).info.lifecycle_stage))

run_id: 37a3fe9b6faf41d89001eca13ad6ca47; lifecycle_stage: active


In [22]:
logged_model = f'runs:/{run_id}/model'

# Load model as a pytorch model, not as the pyfunc model
model = mlflow.pytorch.load_model(logged_model)
model.predict({'This is great news'})


['positive']